In [ ]:
import mlflow
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

from pickle import dump

from scripts.preprocessing_regression_models import Preprocess
from scripts.LinearRegressionTraining import LinearRegressionTraining

from scripts.config_regression import (year_month_train, 
    input_data_path_train,
    seed)

In [ ]:
local_path_save = './local_artifacts_tmp/01_Linear_Regression/'
year_month = year_month_train
input_data_path = input_data_path_train

### MLFlow setting

In [ ]:
if not os.path.exists(local_path_save):
    os.makedirs(local_path_save)

#save all metadata in a sqlite db. Artifacts will be saved on local folder ./mlflow    
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Name of the experiment
exp_name = "01 - Linear Regression"
# set up MlFlow axperiment
experiment_id = mlflow.set_experiment(exp_name)

## Experiments

In [ ]:
run_name = "base"

In [ ]:
linear_regression_training = LinearRegressionTraining(
    input_data_path,
    local_path_save,
    year_month,
    'linear_regression')

In [ ]:
prepr = Preprocess(input_data_path_train)
X, Y = prepr.read_dataframe(request_tgt=True)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=seed)

#preprocessing ohe
shapes_pre = (X_train.shape[0], X_test.shape[0])
X_train_ohe, ohe = prepr.preprocess(df=X_train, fit_ohe=True, drop_first_column=True)
X_test_ohe, _ = prepr.preprocess(df=X_test, fit_ohe=False, drop_first_column=True, ohe=ohe)
assert shapes_pre == (X_train.shape[0], X_test.shape[0])
dump(ohe, open(local_path_save + run_name + '_ohe.pkl', 'wb'))

In [ ]:
best_result = linear_regression_training.objective_lr(X_train=X_train_ohe,
         X_test=X_test_ohe,
         Y_train=Y_train,
         Y_test=Y_test,
         run_name=run_name)

best_result